# Importing the data and creating the data loaders

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
# importing libraries

import glob
import sys
from zipfile import ZipFile 
import concurrent.futures
import gc
from time import time
import cv2

sys.path.insert(0,'../src/')

import PIL as pil

import pandas as pd
import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt

import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
from tqdm import tqdm
import urllib.request
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: emadonev (emadonev-xv-gimnazija) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
def clear_gpu_memory(debug:bool=True):
    """
    Clear GPU memory on all devices.
    Optinonally print the memory before and after clearing.

    Args:
        debug (bool, optional): Print memory before and after clearing. Defaults to False.
    """
    if debug:
        print("GPU memory before clearing.")
        print(f"Allocated: {torch.cuda.memory_allocated() :_}")
        print(f"Reserved: {torch.cuda.memory_reserved() :_}")
    
    # Free up memory
    gc.collect()

    # Clear CUDA memory cache
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print("GPU memory has been cleared.")

    if debug:
        print("GPU memory after clearing.")
        print(f"Allocated: {torch.cuda.memory_allocated() :_}")
        print(f"Reserved: {torch.cuda.memory_reserved() :_}")

In [5]:
clear_gpu_memory()

GPU memory before clearing.
Allocated: 0
Reserved: 0
GPU memory has been cleared.
GPU memory after clearing.
Allocated: 0
Reserved: 0


In [21]:
from data_process import *
from cvt import *
from model_train import *

---

In [ ]:
url = "https://zenodo.org/records/3565489/files/images_gz2.zip?download=1"
urllib.request.urlretrieve(url, "../input/images_gz2.zip")

In [7]:
with ZipFile('../input/images_gz2.zip', 'r') as zip_ref:
    zip_ref.extractall(path='../input/images_gz2/')

---

In [7]:
reference_images = pd.read_csv('../input/filename_mapping.csv')

main_catalogue = pd.read_csv('../input/gz2_classes.csv')

In [8]:
reference_images.head()

,objid,sample,asset_id
0,587722981736120347,original,1
1,587722981736579107,original,2
2,587722981741363294,original,3
3,587722981741363323,original,4
4,587722981741559888,original,5


In [9]:
main_catalogue.head()

,specobjid,dr8objid,dr7objid,ra,dec,rastring,decstring,sample,gz2class,total_classifications,...,t11_arms_number_a36_more_than_4_fraction,t11_arms_number_a36_more_than_4_weighted_fraction,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a36_more_than_4_flag,t11_arms_number_a37_cant_tell_count,t11_arms_number_a37_cant_tell_weight,t11_arms_number_a37_cant_tell_fraction,t11_arms_number_a37_cant_tell_weighted_fraction,t11_arms_number_a37_cant_tell_debiased,t11_arms_number_a37_cant_tell_flag
0,1.802675e+18,NaN,588017703996096547,160.99040,11.703790,10:43:57.70,+11:42:13.6,original,SBb?t,44,...,0.225,0.225,0.225,0,10,10.0,0.250,0.250,0.250,0
1,1.992984e+18,NaN,587738569780428805,192.41083,15.164207,12:49:38.60,+15:09:51.1,original,Ser,45,...,0.000,0.000,0.000,0,0,0.0,0.000,0.000,0.000,0
2,1.489569e+18,NaN,587735695913320507,210.80220,54.348953,14:03:12.53,+54:20:56.2,original,Sc+t,46,...,0.651,0.651,0.651,0,3,3.0,0.070,0.070,0.070,0
3,2.924084e+18,1.237668e+18,587742775634624545,185.30342,18.382704,12:21:12.82,+18:22:57.7,original,SBc(r),45,...,0.071,0.071,0.071,0,6,6.0,0.429,0.429,0.429,0
4,1.387165e+18,1.237658e+18,587732769983889439,187.36679,8.749928,12:29:28.03,+08:44:59.7,extra,Ser,49,...,0.000,0.000,0.000,0,1,1.0,1.000,1.000,1.000,0


In [10]:
# create modified main catalogue
model_01_catalogue = pd.DataFrame()
model_01_catalogue['dr7ID'] = main_catalogue['dr7objid']
model_01_catalogue['class'] = main_catalogue['gz2class']
print(model_01_catalogue.shape)
model_01_catalogue.head()

(243500, 2)


,dr7ID,class
0,588017703996096547,SBb?t
1,587738569780428805,Ser
2,587735695913320507,Sc+t
3,587742775634624545,SBc(r)
4,587732769983889439,Ser


In [11]:
model_01_catalogue.drop(model_01_catalogue[model_01_catalogue['class'] == 'A'].index, inplace=True)
model_01_catalogue.shape

(243253, 2)

In [12]:
# connecting each class with the corresponding asset_id
model_01_catalogue = model_01_catalogue.merge(
    reference_images[['objid', 'asset_id']], 
    left_on='dr7ID', 
    right_on='objid', 
    how='left'
).drop(columns=['objid'])  # Drop extra 'objid' column after merging
model_01_catalogue = model_01_catalogue.sort_values(by=['asset_id']).reset_index(drop=True)

model_01_catalogue['class'] = model_01_catalogue['class'].apply(lambda x: x.replace('(', '').replace(')', '').ljust(6, '0'))
model_01_catalogue.head()

,dr7ID,class,asset_id
0,587722981741363294,Ei0000,3
1,587722981741363323,Sc0000,4
2,587722981741559888,Er0000,5
3,587722981741625481,Er0000,6
4,587722981741625484,Ei0000,7


In [13]:
# creating a label diagram table
label_diagram = pd.DataFrame(columns=['r1', 'r2', 'r3', 'r4', 'r5'])
label_diagram['asset_id'] = model_01_catalogue['asset_id']
label_diagram['r1'] = model_01_catalogue['class'].apply(choose_class1)
label_diagram['r2'] = model_01_catalogue['class'].apply(choose_class2)
label_diagram['r3'] = model_01_catalogue['class'].apply(choose_class3)
label_diagram['r4'] = model_01_catalogue['class'].apply(choose_class4)
label_diagram['r5'] = model_01_catalogue['class'].apply(choose_class5)
label_diagram.head()

,r1,r2,r3,r4,r5,asset_id
0,E,is,0,0,0,3
1,S,c,0,0,0,4
2,E,rs,0,0,0,5
3,E,rs,0,0,0,6
4,E,is,0,0,0,7


---

In [14]:
label_diagram['r1'].value_counts()

r1
E     103515
S      94332
Se     24004
SB     21402
Name: count, dtype: int64

In [15]:
imgs_path = '../input/images_gz2/images/'
W, H, C = 224, 224, 4

In [16]:
file_list = glob.glob(os.path.join(imgs_path, '*.jpg'))
file_list = sorted(file_list)

# select files whose asset_id is the same as the one in the label_diagram
file_list = [f for f in file_list if int(f.split('/')[-1].split('.')[0]) in label_diagram['asset_id'].values]
len(file_list)

243253

In [17]:
file_list = file_list[:30000]

In [24]:
gmorph_d = galaxy_img_dataset(file_list, label_diagram, label_mapping='r1', class_mapping=class_mapping)

In [18]:
class_mapping = {x : i for i, x in enumerate(sorted(label_diagram['r1'].unique()))}

if __name__ == '__main__':
    gmorph_d = galaxy_img_dataset(file_list, label_diagram, label_mapping='r1', class_mapping=class_mapping)

    total_size = len(gmorph_d)
    train_size = int(0.7 * total_size)
    val_size = int(0.2 * total_size)
    test_size = total_size - train_size - val_size

    generator = torch.Generator().manual_seed(42)
    train_dataset, val_dataset, test_dataset = random_split(
        gmorph_d, [train_size, val_size, test_size], generator=generator
    )


---

# Architecture testing

In [19]:
import gc

In [20]:
gc.collect()

0

In [19]:
epochs = 10
lr = 0.0001
tmax = epochs // 3
device= 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
embed_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                            num_workers=16, pin_memory=True
                            )
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=16, pin_memory=True
                        )
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=16, pin_memory=True
                            )

In [ ]:
gmorph_model = CvT(embed_size, len(class_mapping))
optimizer = torch.optim.AdamW(gmorph_model.parameters(), lr=lr, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, tmax, eta_min=0.0001)
loss_func = nn.CrossEntropyLoss()

results, train_pred, valid_pred = train_model(epochs, gmorph_model, train_loader, val_loader, loss_func, optimizer, scheduler, device, save_name=f'test')

In [ ]:
gmorph_model = CvT_patch(embed_size, len(class_mapping))
optimizer = torch.optim.AdamW(gmorph_model.parameters(), lr=lr, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, tmax, eta_min=0.0001)
loss_func = nn.CrossEntropyLoss()

results, results_class, train_pred, train_true, train_probs, valid_pred, valid_true, valid_probs = train_model(epochs, gmorph_model, train_loader, val_loader, loss_func, optimizer, scheduler, device, save_name=f'CvT_patch')

In [20]:
clear_gpu_memory()

GPU memory before clearing.
Allocated: 0
Reserved: 0
GPU memory has been cleared.
GPU memory after clearing.
Allocated: 0
Reserved: 0


In [23]:
gmorph_model = CvT_depth(embed_size, len(class_mapping))
optimizer = torch.optim.AdamW(gmorph_model.parameters(), lr=lr, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, tmax, eta_min=0.0001)
loss_func = nn.CrossEntropyLoss()

results, results_class, train_pred, train_true, train_probs, valid_pred, valid_true, valid_probs = train_model(epochs, gmorph_model, train_loader, val_loader, loss_func, optimizer, scheduler, device, save_name=f'CvT_depth')

Run name: CvT_depth
Epoch 1/10 - Train Loss: 0.9716 - Train Acc: 0.6274 - Valid Loss: 0.8988 - Valid Acc: 0.6688
../output/model_CvT_depth is saved!
Epoch 2/10 - Train Loss: 0.8410 - Train Acc: 0.6835 - Valid Loss: 0.7597 - Valid Acc: 0.7150
../output/model_CvT_depth is saved!
Epoch 3/10 - Train Loss: 0.7947 - Train Acc: 0.6987 - Valid Loss: 0.7464 - Valid Acc: 0.7210
../output/model_CvT_depth is saved!
Epoch 4/10 - Train Loss: 0.7594 - Train Acc: 0.7074 - Valid Loss: 0.7276 - Valid Acc: 0.7200
../output/model_CvT_depth is saved!
Epoch 5/10 - Train Loss: 0.7547 - Train Acc: 0.7109 - Valid Loss: 0.7364 - Valid Acc: 0.7258
../output/model_CvT_depth is saved!
Epoch 6/10 - Train Loss: 0.7279 - Train Acc: 0.7188 - Valid Loss: 0.7116 - Valid Acc: 0.7227
../output/model_CvT_depth is saved!
Epoch 7/10 - Train Loss: 0.7073 - Train Acc: 0.7222 - Valid Loss: 0.8214 - Valid Acc: 0.6865
../output/model_CvT_depth is saved!
Epoch 8/10 - Train Loss: 0.7067 - Train Acc: 0.7223 - Valid Loss: 0.6817 - Va

train_F1,▁▄▅▅▅▆▆▇▇█
train_acc,▁▅▆▆▆▇▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
train_precision,▁▂▂▂▂▇▇▇██
train_recall,▁▄▅▆▆▇▇▇██
valid_F1,▁▃▄▄▄▄▄▆▇█
valid_acc,▁▅▆▅▆▆▃▇█▇
valid_loss,█▄▄▃▃▃▆▂▁▁
valid_precision,▁▂▁▂▁▁▇██▇
valid_recall,▁▃▅▄▅▅▄▆▇█
train_F1,0.61169


Training time = 54.77901005744934 minutes


In [26]:
del gmorph_model
del optimizer
gc.collect()
torch.cuda.empty_cache()

In [27]:
clear_gpu_memory()

GPU memory before clearing.
Allocated: 32_911_360
Reserved: 195_035_136
GPU memory has been cleared.
GPU memory after clearing.
Allocated: 32_911_360
Reserved: 195_035_136


In [21]:
gmorph_model = CvT_heads(embed_size, len(class_mapping))
optimizer = torch.optim.AdamW(gmorph_model.parameters(), lr=lr, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, tmax, eta_min=0.0001)
loss_func = nn.CrossEntropyLoss()

results, results_class, train_pred, train_true, train_probs, valid_pred, valid_true, valid_probs = train_model(epochs, gmorph_model, train_loader, val_loader, loss_func, optimizer, scheduler, device, save_name=f'CvT_heads')

Run name: CvT_heads
Epoch 1/10 - Train Loss: 1.0008 - Train Acc: 0.6218 - Valid Loss: 0.9449 - Valid Acc: 0.6395
../output/model_CvT_heads is saved!
Epoch 2/10 - Train Loss: 0.8177 - Train Acc: 0.6890 - Valid Loss: 0.7056 - Valid Acc: 0.7298
../output/model_CvT_heads is saved!
Epoch 3/10 - Train Loss: 0.7674 - Train Acc: 0.7087 - Valid Loss: 0.7082 - Valid Acc: 0.7262
../output/model_CvT_heads is saved!
Epoch 4/10 - Train Loss: 0.7317 - Train Acc: 0.7202 - Valid Loss: 0.7100 - Valid Acc: 0.7185
../output/model_CvT_heads is saved!
Epoch 5/10 - Train Loss: 0.7113 - Train Acc: 0.7247 - Valid Loss: 0.6593 - Valid Acc: 0.7378
../output/model_CvT_heads is saved!
Epoch 6/10 - Train Loss: 0.6963 - Train Acc: 0.7257 - Valid Loss: 0.6476 - Valid Acc: 0.7357
../output/model_CvT_heads is saved!
Epoch 7/10 - Train Loss: 0.6841 - Train Acc: 0.7294 - Valid Loss: 0.6650 - Valid Acc: 0.7463
../output/model_CvT_heads is saved!
Epoch 8/10 - Train Loss: 0.6700 - Train Acc: 0.7348 - Valid Loss: 0.8028 - Va

train_F1,▁▄▅▆▆▆▇▇▇█
train_acc,▁▅▆▇▇▇▇███
train_loss,█▄▃▃▂▂▂▁▁▁
train_precision,▁▂▃▇▆▇▇▇██
train_recall,▁▅▆▆▆▆▇▇▇█
valid_F1,▁▃▃▃▄▆▇▃▆█
valid_acc,▁▇▆▆▇▇█▅██
valid_loss,█▂▂▂▁▁▁▅▁▂
valid_precision,▁▃▂▃▃▆▇██▇
valid_recall,▁▃▂▂▃▆▇▃▆█
train_F1,0.64281


Training time = 46.505906037489574 minutes


In [20]:
gmorph_model = CvT_stage(embed_size, len(class_mapping))
optimizer = torch.optim.AdamW(gmorph_model.parameters(), lr=lr, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, tmax, eta_min=0.0001)
loss_func = nn.CrossEntropyLoss()

results, results_class, train_pred, train_true, train_probs, valid_pred, valid_true, valid_probs = train_model(epochs, gmorph_model, train_loader, val_loader, loss_func, optimizer, scheduler, device, save_name=f'CvT_stage')

Run name: CvT_stage
Epoch 1/10 - Train Loss: 1.0269 - Train Acc: 0.6002 - Valid Loss: 0.8943 - Valid Acc: 0.6572
../output/model_CvT_stage is saved!
Epoch 2/10 - Train Loss: 0.8506 - Train Acc: 0.6792 - Valid Loss: 0.8212 - Valid Acc: 0.6893
../output/model_CvT_stage is saved!
Epoch 3/10 - Train Loss: 0.7837 - Train Acc: 0.7026 - Valid Loss: 0.7138 - Valid Acc: 0.7233
../output/model_CvT_stage is saved!
Epoch 4/10 - Train Loss: 0.7679 - Train Acc: 0.7091 - Valid Loss: 0.8063 - Valid Acc: 0.6735
../output/model_CvT_stage is saved!
Epoch 5/10 - Train Loss: 0.7391 - Train Acc: 0.7194 - Valid Loss: 0.6930 - Valid Acc: 0.7353
../output/model_CvT_stage is saved!
Epoch 6/10 - Train Loss: 0.7258 - Train Acc: 0.7192 - Valid Loss: 0.7046 - Valid Acc: 0.7257
../output/model_CvT_stage is saved!
Epoch 7/10 - Train Loss: 0.7116 - Train Acc: 0.7272 - Valid Loss: 0.6686 - Valid Acc: 0.7453
../output/model_CvT_stage is saved!
Epoch 8/10 - Train Loss: 0.6970 - Train Acc: 0.7290 - Valid Loss: 0.7547 - Va

train_F1,▁▆▇▇▇▇████
train_acc,▁▅▆▇▇▇████
train_loss,█▅▃▃▂▂▂▁▁▁
train_precision,▁▂▂▂▂▂▂▂▂█
train_recall,▁▅▇▇▇▇████
valid_F1,▁▅▇▅▇▇█▇██
valid_acc,▁▄▆▂▇▆█▆██
valid_loss,█▆▃▆▂▃▂▄▂▁
valid_precision,▅▁▄▅▇█▆▂▅▆
valid_recall,▁▅▇▅▆▆▇███
train_F1,0.56474


Training time = 27.99525281985601 minutes


---

In [19]:
epochs = 20
lr = 0.0001
tmax = epochs
device= 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
embed_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                            num_workers=16, pin_memory=True
                            )
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=16, pin_memory=True
                        )
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=16, pin_memory=True
                            )

In [23]:
gmorph_model = CvT_gmorph(embed_size, len(class_mapping))
optimizer = torch.optim.AdamW(gmorph_model.parameters(), lr=lr, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, tmax, eta_min=0.0001)
loss_func = nn.CrossEntropyLoss()

results, results_class, train_pred, train_true, train_probs, valid_pred, valid_true, valid_probs = train_model(epochs, gmorph_model, train_loader, val_loader, loss_func, optimizer, scheduler, device, save_name=f'CvT_gmorph')

Run name: CvT_gmorph
Epoch 1/20 - Train Loss: 1.1102 - Train Acc: 0.5346 - Valid Loss: 0.9626 - Valid Acc: 0.6283
../output/model_CvT_gmorph is saved!
Epoch 2/20 - Train Loss: 0.8774 - Train Acc: 0.6717 - Valid Loss: 0.7731 - Valid Acc: 0.7063
../output/model_CvT_gmorph is saved!
Epoch 3/20 - Train Loss: 0.7932 - Train Acc: 0.7006 - Valid Loss: 0.8173 - Valid Acc: 0.6725
../output/model_CvT_gmorph is saved!
Epoch 4/20 - Train Loss: 0.7540 - Train Acc: 0.7152 - Valid Loss: 0.9673 - Valid Acc: 0.6183
../output/model_CvT_gmorph is saved!
Epoch 5/20 - Train Loss: 0.7319 - Train Acc: 0.7212 - Valid Loss: 0.8393 - Valid Acc: 0.6818
../output/model_CvT_gmorph is saved!
Epoch 6/20 - Train Loss: 0.7205 - Train Acc: 0.7270 - Valid Loss: 0.6987 - Valid Acc: 0.7375
../output/model_CvT_gmorph is saved!
Epoch 7/20 - Train Loss: 0.7013 - Train Acc: 0.7336 - Valid Loss: 0.7121 - Valid Acc: 0.7300
../output/model_CvT_gmorph is saved!
Epoch 8/20 - Train Loss: 0.6918 - Train Acc: 0.7333 - Valid Loss: 0.6

KeyboardInterrupt: 